In [56]:
import numpy as np
import pandas as pd
import os
import argparse
from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split
import time

In [2]:
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
from keras.layers import Input, concatenate, Embedding, Reshape
from keras.layers import Merge, Flatten, merge, Lambda, Dropout
from keras.layers.normalization import BatchNormalization
from keras.models import Model
from keras.regularizers import l2, l1_l2
import tensorflow as tf

/Users/BharathiSrinivasan/anaconda2/envs/python36/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [20]:
myfolder = '/Users/BharathiSrinivasan/Documents/HU-MEMS-Sem3/Info_Systems/repo/Recommender_DNN/Data/'
train = pd.read_csv(myfolder + 'bigtrain.csv')
test = pd.read_csv(myfolder + 'bigtest.csv')

#To Do : Include this in pre-processing dataset while creating bigtrain and bigtest
#train.dropna()
#test.dropna()

,order_id,product_id,add_to_cart_order,reordered,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,aisle_id,department_id
0,2445303,49302,2,1,199120,1,49,3,20,7,120,16
1,3231517,49302,6,1,63189,1,42,2,13,7,120,16
2,214201,11109,3,1,131593,1,42,6,14,6,108,16
3,332153,11109,3,1,29758,1,38,1,14,8,108,16
4,583113,11109,8,1,4486,1,35,4,8,24,108,16
5,687429,11109,4,1,23092,1,51,1,11,12,108,16
6,739022,11109,26,0,103264,1,28,0,9,12,108,16
7,763067,11109,8,1,171666,1,49,4,11,6,108,16
8,883854,11109,2,1,89088,1,44,6,11,0,108,16
9,995322,11109,11,0,151424,1,28,0,14,16,108,16


In [42]:
train['IS_TRAIN'] = 1
test['IS_TRAIN'] = 0
#df = pd.concat([train, test],ignore_index=True)
df = train

In [43]:
#PRE-PROCESSING
y = df['reordered'].values
df.pop('reordered')

0           0
1           1
2           1
3           1
4           1
5           1
6           1
7           1
8           1
9           1
10          1
11          1
12          1
13          1
14          1
15          1
16          1
17          1
18          1
19          1
20          1
21          1
22          1
23          1
24          1
25          1
26          1
27          1
28          1
29          1
           ..
16587477    1
16587478    1
16587479    1
16587480    1
16587481    1
16587482    1
16587483    1
16587484    0
16587485    1
16587486    1
16587487    0
16587488    0
16587489    0
16587490    0
16587491    0
16587492    0
16587493    0
16587494    0
16587495    0
16587496    1
16587497    0
16587498    0
16587499    1
16587500    0
16587501    0
16587502    0
16587503    0
16587504    0
16587505    0
16587506    0
Name: reordered, Length: 16587507, dtype: int64

In [44]:
CATEGORICAL_COLUMNS = ["order_dow", "order_hour_of_day"]
CONTINUOUS_COLUMNS = [ "user_orders", "order_number", "add_to_cart_order"]

#One-hot encoding categorical columns
df = pd.get_dummies(df, columns=[x for x in CATEGORICAL_COLUMNS])

In [45]:
#To Do : Include this in pre-processing dataset while creating bigtrain and bigtest
df.drop(['eval_set'],axis=1)

,user_id,user_orders,order_id,order_number,days_since_prior_order,product_id,add_to_cart_order,aisle_id,department_id,IS_TRAIN,...,order_hour_of_day_14,order_hour_of_day_15,order_hour_of_day_16,order_hour_of_day_17,order_hour_of_day_18,order_hour_of_day_19,order_hour_of_day_20,order_hour_of_day_21,order_hour_of_day_22,order_hour_of_day_23
0,17,40,1737705,1,30,7350,1,115,7,1,...,0,0,0,0,0,0,0,0,0,0
1,17,40,1681401,2,3,7350,1,115,7,1,...,0,0,0,0,0,0,0,0,0,0
2,17,40,2680214,3,5,7350,1,115,7,1,...,0,0,0,0,0,0,0,0,0,0
3,17,40,3237467,5,5,7350,1,115,7,1,...,0,0,0,1,0,0,0,0,0,0
4,17,40,2616505,6,5,7350,2,115,7,1,...,0,0,0,1,0,0,0,0,0,0
5,17,40,2648319,7,4,7350,1,115,7,1,...,0,0,0,0,0,0,0,0,0,0
6,17,40,2430354,8,6,7350,1,115,7,1,...,0,1,0,0,0,0,0,0,0,0
7,17,40,900554,10,4,7350,1,115,7,1,...,0,1,0,0,0,0,0,0,0,0
8,17,40,805025,11,4,7350,2,115,7,1,...,0,1,0,0,0,0,0,0,0,0
9,17,40,912404,12,5,7350,1,115,7,1,...,1,0,0,0,0,0,0,0,0,0


In [46]:
df[df==np.inf]=np.nan
df.dropna()
np.all(np.isfinite(df))
#df[np.isfinite(df) | np.isnan(df)]

True

In [47]:
df.reset_index()
df = pd.DataFrame(MinMaxScaler().fit_transform(df), columns=df.columns)
X = df.values

In [50]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [51]:
#Wide: Linear Model
wide = Sequential()
wide.add(Dense(1, input_dim=X_train.shape[1]))

In [52]:
#Deep Neural Network with 2 hidden layers of 100 and 50 neurons
deep = Sequential()
# TODO: add embedding
deep.add(Dense(input_dim=X_train.shape[1], output_dim=100, activation='relu'))
deep.add(Dense(100, activation='relu'))
deep.add(Dense(50, activation='relu'))
deep.add(Dense(1, activation='sigmoid'))

/Users/BharathiSrinivasan/anaconda2/envs/python36/lib/python3.6/site-packages/ipykernel_launcher.py:3: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(input_dim=42, activation="relu", units=100)`
  This is separate from the ipykernel package so we can avoid doing imports until


In [54]:
#Wide + Deep
model = Sequential()
model.add(Merge([wide, deep], mode='concat', concat_axis=1))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['accuracy'])

/Users/BharathiSrinivasan/anaconda2/envs/python36/lib/python3.6/site-packages/ipykernel_launcher.py:3: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  This is separate from the ipykernel package so we can avoid doing imports until


In [58]:
t0=time.time()
model.fit([X_train, X_train], y_train, nb_epoch=10, batch_size=32)
print("training time:", round(time.time()-t0, 3), "s")

/Users/BharathiSrinivasan/anaconda2/envs/python36/lib/python3.6/site-packages/keras/models.py:944: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Epoch 1/10
12440630/12440630 [==============================] - 902s 73us/step - loss: 0.5346 - acc: 0.7506
Epoch 2/10
12440630/12440630 [==============================] - 867s 70us/step - loss: 0.5368 - acc: 0.7512
Epoch 3/10
12440630/12440630 [==============================] - 803s 65us/step - loss: 0.5388 - acc: 0.7511
Epoch 4/10
12440630/12440630 [==============================] - 836s 67us/step - loss: 0.5400 - acc: 0.7510
Epoch 5/10
12440630/12440630 [==============================] - 800s 64us/step - loss: 0.5420 - acc: 0.7506
Epoch 6/10
12440630/12440630 [==============================] - 814s 65us/step - loss: 0.5432 - acc: 0.7502
Epoch 7/10
12440630/12440630 [==============================] - 832s 67us/step - loss: 0.5455 - acc: 0.7500
Epoch 8/10
12440630/12440630 [==============================] - 860s 69us/step - loss: 0.5463 - acc: 0.7498
Epoch 9/10
12440630/12440630 [==============================] - 840s 68us/step - loss: 0.5477 - acc: 0.7489
Epoch 10/10
12440630/1244063

In [59]:
loss, accuracy = model.evaluate([X_test, X_test], y_test)
print('\n', 'test accuracy:', accuracy)

4146877/4146877 [==============================] - 121s 29us/step

 test accuracy: 0.7483245825715008


In [60]:
# serialize model to JSON
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("wide_deep1.h5")
print("Saved model to disk")

Saved model to disk


In [62]:
""""
#Later
# load json and create model
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("model.h5")
print("Loaded model from disk")
"""""

'"\n#Later\n# load json and create model\njson_file = open(\'model.json\', \'r\')\nloaded_model_json = json_file.read()\njson_file.close()\nloaded_model = model_from_json(loaded_model_json)\n# load weights into new model\nloaded_model.load_weights("model.h5")\nprint("Loaded model from disk")\n'